In [2]:
from tsp_solver.greedy import solve_tsp
import torch
page_embeddings = torch.randn(100, 512)

# tsp


In [3]:
from tsp_solver.greedy import solve_tsp
distance_matrix = 1 - torch.matmul(page_embeddings, page_embeddings.T)
path = solve_tsp(distance_matrix, endpoints=(0, page_embeddings.shape[0]-1))
print(path)

[0, 93, 2, 64, 72, 76, 25, 21, 43, 61, 97, 18, 45, 1, 68, 63, 74, 50, 40, 49, 10, 91, 5, 71, 95, 96, 28, 88, 59, 73, 55, 30, 98, 89, 78, 85, 83, 32, 52, 3, 44, 12, 67, 6, 13, 14, 19, 82, 54, 46, 69, 48, 94, 70, 57, 4, 16, 60, 20, 66, 75, 17, 7, 39, 34, 37, 23, 24, 87, 22, 86, 90, 62, 58, 53, 11, 29, 41, 9, 38, 81, 27, 15, 65, 47, 51, 79, 42, 8, 33, 31, 84, 35, 56, 36, 92, 26, 80, 77, 99]


In [4]:
page_embeddings.shape


torch.Size([100, 512])

In [5]:
page_embeddings[:4]

tensor([[-1.1046, -1.2527, -0.1553,  ..., -0.8748, -2.3633, -0.7665],
        [-0.9851,  0.5123,  0.2101,  ..., -0.7087, -1.5597, -2.7528],
        [-0.0089, -1.2958,  1.3167,  ...,  0.7459, -1.3162, -0.7684],
        [-0.9076,  0.5373, -0.7236,  ..., -1.8741, -0.1846,  0.7369]])

In [7]:
index = 2
page_embeddings[torch.arange(page_embeddings.shape[0]) != index][:4]

tensor([[-1.1046, -1.2527, -0.1553,  ..., -0.8748, -2.3633, -0.7665],
        [-0.9851,  0.5123,  0.2101,  ..., -0.7087, -1.5597, -2.7528],
        [-0.9076,  0.5373, -0.7236,  ..., -1.8741, -0.1846,  0.7369],
        [ 0.5716,  1.9660,  0.4616,  ...,  0.7688, -1.1503,  0.1121]])